In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [3]:
movies = pd.read_csv('dataset/autoencoders/ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('dataset/autoencoders/ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('dataset/autoencoders/ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [4]:
training_set = pd.read_csv('dataset/autoencoders/ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('dataset/autoencoders/ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')


In [5]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

In [6]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [8]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [9]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [10]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [11]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimizer.step()
  print('epoch: '+str(epoch)+'loss: '+ str(train_loss/s))

epoch: 1loss: tensor(1.7711)
epoch: 2loss: tensor(1.0966)
epoch: 3loss: tensor(1.0533)
epoch: 4loss: tensor(1.0383)
epoch: 5loss: tensor(1.0309)
epoch: 6loss: tensor(1.0268)
epoch: 7loss: tensor(1.0239)
epoch: 8loss: tensor(1.0219)
epoch: 9loss: tensor(1.0207)
epoch: 10loss: tensor(1.0196)
epoch: 11loss: tensor(1.0189)
epoch: 12loss: tensor(1.0181)
epoch: 13loss: tensor(1.0180)
epoch: 14loss: tensor(1.0176)
epoch: 15loss: tensor(1.0172)
epoch: 16loss: tensor(1.0171)
epoch: 17loss: tensor(1.0168)
epoch: 18loss: tensor(1.0163)
epoch: 19loss: tensor(1.0166)
epoch: 20loss: tensor(1.0161)
epoch: 21loss: tensor(1.0161)
epoch: 22loss: tensor(1.0160)
epoch: 23loss: tensor(1.0159)
epoch: 24loss: tensor(1.0156)
epoch: 25loss: tensor(1.0155)
epoch: 26loss: tensor(1.0155)
epoch: 27loss: tensor(1.0150)
epoch: 28loss: tensor(1.0150)
epoch: 29loss: tensor(1.0126)
epoch: 30loss: tensor(1.0114)
epoch: 31loss: tensor(1.0092)
epoch: 32loss: tensor(1.0092)
epoch: 33loss: tensor(1.0057)
epoch: 34loss: tens

In [12]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9548)
